<h1 align="center">Traffic Control</h1>

## Steps Involved

### Step1: Yolo Object Detection(cars,bikes,cycle,trucks,auto)

### Step2: Ratio of cars to area involoved during object detection

### Step3: Interface Implementations

### Step4 : Connecting Everything


## Step 1 : Yolo Object detection

In [47]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import time
import os

In [96]:
# load the COCO class labels our YOLO model was trained on
labelsPath = os.path.sep.join(["weights", "coco.names"])
LABELS = open(labelsPath).read().strip().split("\n")

# initialize a list of colors to represent each possible class label
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),dtype="uint8")
config_file="/home/shashank/Desktop/traffic_control/weights/yolov3.cfg"

weights_path="/home/shashank/Desktop/traffic_control/weights/yolov3.weights"
net=cv2.dnn.readNetFromDarknet(config_file,weights_path)


In [97]:
def detect(gray,frame):
    (H, W) = frame.shape[:2]
    # determine only the *output* layer names that we need from YOLO
    ln = net.getLayerNames()
    ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    # construct a blob from the input image and then perform a forward
    # pass of the YOLO object detector, giving us our bounding boxes and
    # associated probabilities
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),swapRB=True, crop=False)
    net.setInput(blob)
    layerOutputs = net.forward(ln)
    # initialize our lists of detected bounding boxes, confidences, and
    # class IDs, respectively
    boxes = []
    confidences = []
    classIDs = []
    counts=[]
    for output in layerOutputs:
        # loop over each of the detections
        for detection in output:
            # extract the class ID and confidence (i.e., probability) of
            # the current object detection
            scores = detection[5:]
            classID = np.argmax(scores)
            if classID in [1,2,3,5,7]:
                confidence = scores[classID]
                # filter out weak predictions by ensuring the detected
                # probability is greater than the minimum probability
                if confidence > 0.5:
                    # scale the bounding box coordinates back relative to the
                    # size of the image, keeping in mind that YOLO actually
                    # returns the center (x, y)-coordinates of the bounding
                    # box followed by the boxes' width and height
                    box = detection[0:4] * np.array([W, H, W, H])
                    (centerX, centerY, width, height) = box.astype("int")
                    # use the center (x, y)-coordinates to derive the top and
                    # and left corner of the bounding box
                    x = int(centerX - (width / 2))
                    y = int(centerY - (height / 2))
                    # update our list of bounding box coordinates, confidences,
                    # and class IDs
                    boxes.append([x, y, int(width), int(height)])
                    confidences.append(float(confidence))
                    classIDs.append(classID)
                    counts.append(1)
                    
        idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.5,0.3)
        
        # ensure at least one detection exists
        
        if len(idxs) > 0:
            # loop over the indexes we are keeping
            for i in idxs.flatten():
                # extract the bounding box coordinates
                (x, y) = (boxes[i][0], boxes[i][1])
                (w, h) = (boxes[i][2], boxes[i][3])
                # draw a bounding box rectangle and label on the frame
                color = [int(c) for c in COLORS[classIDs[i]]]
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                text = "{}: {:.4f}".format(LABELS[classIDs[i]],
                    confidences[i])
                cv2.putText(frame,text,(x, y - 5),cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                
        # apply non-maxima suppression to suppress weak, overlapping
        # bounding boxes
        idxs = cv2.dnn.NMSBoxes(boxes, confidences,0.5,0.3)
        # ensure at least one detection exists
        if len(idxs) > 0:
            # loop over the indexes we are keeping
            for i in idxs.flatten():
                # extract the bounding box coordinates
                (x, y) = (boxes[i][0], boxes[i][1])
                (w, h) = (boxes[i][2], boxes[i][3])
                # draw a bounding box rectangle and label on the frame
                color = [int(c) for c in COLORS[classIDs[i]]]
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                text = "{}: {:.4f}".format(LABELS[classIDs[i]],confidences[i],len(counts))
                cv2.putText(frame, text, (x, y - 5),cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                if len(counts)//167>0.12 and len(counts)//167<0.14:
                    a="Heavy Traffic Ahead"
                    perc="90%"
                elif len(counts)//167<0.12 and len(counts)//167>0.8:
                    a="Drive Safely"
                    perc="75%"
                else:
                    a=" You are free to go"
                    perc="40%"
                text1="{}".format(a,perc)
                cv2.putText(frame,text1,(x,y+20),cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            
        return frame

In [99]:
cap=cv2.VideoCapture("http://192.168.43.42:8080/video")
while True:
    ret,frame=cap.read()
    frame=cv2.resize(frame,(940,640),interpolation=cv2.INTER_LINEAR)
    #frame=cv2.flip(frame,1)
    if ret:
        gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        canvas=detect(gray,frame)
        cv2.imshow("image",canvas)
        if cv2.waitKey(1)&0xFF==ord("q"):
            cv2.destroyAllWindows()
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()